In [ ]:
import visdom
import numpy as np
import sys
sys.path.append("../TimeSeriesTools")
import utils
from cerberus import Validator
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.express as px
import plotly.tools as tls

import pandas as pd
from datetime import datetime


def find_all_data(col):
    data_list=[]
    for d in collection.find():
        data_list.append(d)
    print(len(data_list), " documents found")
    return data_list  

In [ ]:
def to_unix_time(dt):
    epoch =  datetime.utcfromtimestamp(7200)
    return (dt - epoch).total_seconds() * 1000

In [ ]:
def clean_data(data):
    schema =  {
    "_id":{'required': True},
    "Heure":{'type': 'string','required': True,'empty': False},
    "Temps écoulé":{'type': 'string','required': True,'empty': False},
    "Latitude":{'type': 'string','required': True,'empty': False},
    "Longitude":{'type': 'string','required': True,'empty': False},
    "Altitude":{'type': 'string','required': True,'empty': False},
    "Head_Rel_True North":{'type': 'string','required': True,'empty': False},
    "Pressure":{'type': 'string','required': True,'empty': False},
    "Temperature":{'type': 'string','required': True,'empty': False},
    "Humidity":{'type': 'string','required': True,'empty': False},
    "MDA Wnd Dir":{'type': 'string','required': True,'empty': False},
    "MDA Wnd Speed":{'type': 'string','required': True,'empty': False},
    "MWD Wind Dir":{'type': 'string','required': True,'empty': False},
    "MWD Wind Speed":{'type': 'string','required': True,'empty': False},
    "CavityPressure":{'type': 'string','required': True,'empty': False},    
    "CavityTemp":{'type': 'string','required': True,'empty': False},
    "CH4":{'type': 'string','required': True,'empty': False},
    "CH4_dry":{'type': 'string','required': True,'empty': False},
    "C2H6":{'type': 'string','required': True,'empty': False},
    "C2H6_dry":{'type': 'string','required': True,'empty': False},
    "13CH4":{'type': 'string','required': True,'empty': False},
    "H2O":{'type': 'string','required': True,'empty': False},
    "CO2":{'type': 'string','required': True,'empty': False},
    "C2C1Ratio":{'type': 'string','required': True,'empty': False},
    "Delta_iCH4_Raw":{'type': 'string','required': True,'empty': False},
    "HP_Delta_iCH4_30s":{'type': 'string','required': True,'empty': False},
    "HP_Delta_iCH4_2min":{'type': 'string','required': True,'empty': False},
    "HP_Delta_iCH4_5min":{'type': 'string','required': True,'empty': False}
    }
    v = Validator(schema)
    for index,item in enumerate(data,start=0):
        res = v.validate(item)
        if (res == False):
            print("corrupt data in line :",index,", error : ",v.errors)
            del data[index]

In [ ]:
domain = "localhost"
port=27017
client = utils.mongodb_connect(domain, port)
collection = client.test['eolienne_DT']
data = find_all_data(collection)

In [ ]:
clean_data(data)

In [ ]:
df = pd.DataFrame(data)
df

In [ ]:
dates_list = [datetime.strptime(date, '%d/%m/%Y %H:%M:%S') for date in df['Heure']]
dates_list

In [ ]:
x_ref = dates_list[0:5000]
x=[str(xx) for xx in x_ref]
fig = go.Figure()
fig.add_trace(go.Scatter(
                x=x,
                y=[float(item.replace(',', '.')) for item in df["CH4"][0:5000]],
                name="CH4",
                line_color='deepskyblue',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=x,
                y=[float(item.replace(',', '.')) for item in df["Humidity"][0:5000]],
                name="Humidity",
                line_color='dimgray',
                opacity=0.8))

# Use date string to set xaxis range
fig.update_layout(xaxis_range=[to_unix_time(x_ref[0]),
                               to_unix_time(x_ref[-1])],
                  title_text="Manually Set Date Range")
fig.show()

In [ ]:
vis = visdom.Visdom()
vis.plotlyplot(fig)